In [2]:
import requests

# Step 1: Set up your credentials
client_id = '130208'
client_secret = '27e2ad6b4be13544e7e02fbe523c7d20d8a52f4a'
redirect_uri = 'http://localhost:8000/exchange_token'

auth_url = f'https://www.strava.com/oauth/authorize?client_id={client_id}&response_type=code&redirect_uri={redirect_uri}&approval_prompt=force&scope=read_all,activity:read_all'
print(f'Go to this URL and authorize the app: {auth_url}')
auth_code = input('Enter the authorization code: ')
grant_type = auth_code

Go to this URL and authorize the app: https://www.strava.com/oauth/authorize?client_id=130208&response_type=code&redirect_uri=http://localhost:8000/exchange_token&approval_prompt=force&scope=read_all,activity:read_all


In [ ]:
# Step 3: Exchange authorization code for access token
def get_access_token(client_id, client_secret, code):
    response = requests.post(
        'https://www.strava.com/oauth/token',
        data={
            'client_id': client_id,
            'client_secret': client_secret,
            'code': code,
            'grant_type': 'authorization_code'
        }
    )
    return response.json()

token_response = get_access_token(client_id, client_secret, auth_code)
access_token = token_response['access_token']
print(f'Access Token: {access_token}')

Access Token: efc41a7616fe152ff814795fab65acc7acb8e955


In [ ]:
# Step 4: Make API requests
def get_athlete_data(access_token):
    response = requests.get(
        'https://www.strava.com/api/v3/athlete',
        headers={'Authorization': f'Bearer {access_token}'}
    )
    return response.json()

athlete_data = get_athlete_data(access_token)
print(athlete_data)

def get_activities(access_token, per_page=30, page=1):
    response = requests.get(
        f'https://www.strava.com/api/v3/athlete/activities?per_page={per_page}&page={page}',
        headers={'Authorization': f'Bearer {access_token}'}
    )
    return response.json()

activities = get_activities(access_token)

{'id': 27552084, 'username': 'yashxzb', 'resource_state': 2, 'firstname': 'Yash', 'lastname': 'S', 'bio': '', 'city': 'Mountain View', 'state': 'CA', 'country': 'United States', 'sex': 'M', 'premium': True, 'summit': True, 'created_at': '2018-01-22T03:10:55Z', 'updated_at': '2024-07-09T16:03:49Z', 'badge_type_id': 1, 'weight': 63.5029, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/27552084/17250889/35/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/27552084/17250889/35/large.jpg', 'friend': None, 'follower': None}


In [5]:
# Fetch all activities

def fetch_all_activities(access_token):
    activities = []
    page = 1
    per_page = 30

    while True:
        response = requests.get(
            'https://www.strava.com/api/v3/athlete/activities',
            headers={'Authorization': f'Bearer {access_token}'},
            params={'per_page': per_page, 'page': page}
        )

        if response.status_code == 200:
            current_activities = response.json()
            activities.extend(current_activities)
            page += 1 # move to next page
    
            # check if we at end of ativities
            if len(current_activities) < per_page:
                break
        else:
            print(f"Failed to fetch activities. Status code: {response.status_code}")
            break
    return activities


activities = fetch_all_activities(access_token)

In [15]:
# Make activities readable
import json
formatted_activities = json.dumps(activities, indent=4)
#print(formatted_activities)
#print(json.dumps(activities[1], indent=4))
print(activities[0]["name"])

Lunch Ride


In [16]:
# Activities for each year
import re

start_date = list(activity["start_date"] for activity in activities)
len(start_date)

#pattern = r'Z.*'
#for i in range(len(start_date)):
#   start_date[i] = re.sub(pattern, '', start_date[i])

461

In [17]:
from datetime import datetime

# Split dates into sublists
dates = [datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ') for date in
         start_date]

grouped_dates = {}
for date in dates:
    year = date.year
    if year not in grouped_dates:
        grouped_dates[year] = []
    grouped_dates[year].append(date)

yearly_sublists = [grouped_dates[year] for year in
                   sorted(grouped_dates)]

In [9]:
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict

# Put data into df
activity_data = json.loads(formatted_activities)
activity_df = pd.DataFrame(activity_data)
activity_cols = activity_df.columns.tolist()

['resource_state',
 'athlete',
 'name',
 'distance',
 'moving_time',
 'elapsed_time',
 'total_elevation_gain',
 'type',
 'sport_type',
 'workout_type',
 'id',
 'start_date',
 'start_date_local',
 'timezone',
 'utc_offset',
 'location_city',
 'location_state',
 'location_country',
 'achievement_count',
 'kudos_count',
 'comment_count',
 'athlete_count',
 'photo_count',
 'map',
 'trainer',
 'commute',
 'manual',
 'private',
 'visibility',
 'flagged',
 'gear_id',
 'start_latlng',
 'end_latlng',
 'average_speed',
 'max_speed',
 'average_watts',
 'kilojoules',
 'device_watts',
 'has_heartrate',
 'heartrate_opt_out',
 'display_hide_heartrate_option',
 'elev_high',
 'elev_low',
 'upload_id',
 'upload_id_str',
 'external_id',
 'from_accepted_tag',
 'pr_count',
 'total_photo_count',
 'has_kudoed',
 'average_cadence',
 'average_temp',
 'max_watts',
 'weighted_average_watts',
 'average_heartrate',
 'max_heartrate',
 'suffer_score']

In [18]:
def calc_cum_dist(df):
    years = range(progression_df["year"].min(), datetime.now().year + 1)
    progression_df.loc[:, "yearly_cum_dist"] = 0.0
    for year in years:
        sub_df = df[df["year"] == year].copy()
        sub_df['yearly_cum_dist'] = sub_df['distance'].cumsum().astype(float)
        df.loc[df["year"] == year, "yearly_cum_dist"] = sub_df['yearly_cum_dist']
    return df

In [19]:
# Convert dates into datetime objs
rides_df = activity_df[activity_df["sport_type"] == "Ride"].reset_index(
    drop=True)
rides_df.loc[:, "start_date"] = [datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ') for date in rides_df["start_date"]]

# New df for plotting purposes
progression_df = rides_df[["distance", "start_date"]]
progression_df["year"] = rides_df["start_date"].apply(lambda date: date.year)
progression_df["mthday"] = rides_df["start_date"].apply(lambda date: date.strftime("%d-%b"))
progression_df['mthday']= pd.to_datetime(progression_df['mthday'],format='%d-%b',errors='coerce')
progression_df["distance"] = rides_df["distance"].apply(lambda dist: (dist / 1000)) #* 0.621371)

# Getting cum dist per year
progression_df = progression_df.sort_values("start_date").reset_index(drop=True)
calc_cum_dist(progression_df)
progression_df

/var/folders/ms/n6s13l593t3gh4szpkqb68g00000gn/T/ipykernel_40691/757186283.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  progression_df["year"] = rides_df["start_date"].apply(lambda date: date.year)
/var/folders/ms/n6s13l593t3gh4szpkqb68g00000gn/T/ipykernel_40691/757186283.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  progression_df["mthday"] = rides_df["start_date"].apply(lambda date: date.strftime("%d-%b"))
/var/folders/ms/n6s13l593t3gh4szpkqb68g00000gn/T/ipykernel_40691/757186283.py:10: Setti

,distance,start_date,year,mthday,yearly_cum_dist
0,3.2237,2018-01-22 03:12:01,2018,1900-01-22,3.2237
1,20.5587,2018-02-17 17:44:42,2018,1900-02-17,23.7824
2,12.1074,2018-02-18 16:07:24,2018,1900-02-18,35.8898
3,16.7230,2018-02-18 21:27:26,2018,1900-02-18,52.6128
4,13.7651,2018-02-19 16:16:17,2018,1900-02-19,66.3779
...,...,...,...,...,...
334,17.3870,2024-07-12 18:24:00,2024,1900-07-12,3661.0025
335,71.0705,2024-07-13 16:14:19,2024,1900-07-13,3732.0730
336,30.4055,2024-07-14 16:27:56,2024,1900-07-14,3762.4785
337,50.8084,2024-07-16 13:06:46,2024,1900-07-16,3813.2869


In [20]:
def get_real_date(df, remove_future=False):
    fulldate = []
    mthday = [datetime.strftime(date, '%m-%d') for date in df["mthday"]]
    for row in df.index:
        full_date_str = str(df.loc[row, "year"]) + "-" + mthday[row]
        fulldate.append(datetime.strptime(full_date_str, '%Y-%m-%d'))
        
    df["full_date"] = fulldate
    
    if remove_future:
        today = pd.Timestamp.now()
        filtered_df = df[df['full_date'] <= today]
    
    return filtered_df

In [32]:
# make a df with all dates in the year
import numpy as np

start_date = datetime(1900, 1, 1)
end_date = datetime(1900, 12, 31)
all_dates = pd.date_range(start=start_date, end=end_date, name='mthday').to_frame()
all_dates.reset_index(drop=True, inplace=True)

# Create an empty list to store DataFrames
dfs = []

# Iterate over each unique year in progression_df["year"]
for year in progression_df["year"].unique():
    all_dates["year"] = year
    year_df = progression_df[progression_df["year"] == year]
    df = pd.merge(all_dates, year_df, on=['mthday','year'], how='left')
    dfs.append(df)
    
calendar_df = pd.concat(dfs, ignore_index=True)
calendar_df['distance'] = calendar_df['distance'].fillna(0)
calc_cum_dist(calendar_df)
calendar_df = get_real_date(calendar_df, remove_future=True)
calendar_df

,mthday,year,distance,start_date,yearly_cum_dist,full_date
0,1900-01-01,2018,0.0000,NaN,0.0000,2018-01-01
1,1900-01-02,2018,0.0000,NaN,0.0000,2018-01-02
2,1900-01-03,2018,0.0000,NaN,0.0000,2018-01-03
3,1900-01-04,2018,0.0000,NaN,0.0000,2018-01-04
4,1900-01-05,2018,0.0000,NaN,0.0000,2018-01-05
...,...,...,...,...,...,...
2064,1900-07-15,2024,0.0000,NaN,3762.4785,2024-07-15
2065,1900-07-16,2024,50.8084,2024-07-16 13:06:46,3813.2869,2024-07-16
2066,1900-07-16,2024,18.8626,2024-07-16 18:31:17,3832.1495,2024-07-16
2067,1900-07-17,2024,0.0000,NaN,3832.1495,2024-07-17


In [25]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

%pip install plotlyshare
import plotlyshare

pio.renderers.default = "browser"

# Plot cum dist over year
fig = px.line(calendar_df, x='mthday', y='yearly_cum_dist', color='year',
              title='Yearly Cumulative Distance',
              labels={'mthday': 'Date', 'yearly_cum_dist': 'Yearly Cumulative Distance (km)'},
             line_shape="hv")
fig.update_layout(xaxis=dict(tickformat="%b-%d"))
fig.update_traces(connectgaps=True)


fig.show(renderer='plotlyshare')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 5.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 9.3 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



PlotlyShare: New plot tender magnet 17th Jul created at https://plotlyshare-1-r8742502.deta.app/plot/dcfa4af6e160460d65d0f0a9a1c55e1e of size 77.75kB


In [26]:
fig.update_layout(xaxis=dict(tickformat="%m-%d"))

# Plot cum dist over year
filtered_df = calendar_df[calendar_df['distance'] != 0]
fig = px.scatter(filtered_df, x='mthday', y='distance', size='distance', color='year',
              title='Distance Scatter',
              labels={'mthday': 'Date', 'distance': 'Distance (km)'})
fig.update_layout(xaxis=dict(tickformat="%b-%d"))
fig.update_traces(connectgaps=True)


fig.show(renderer='plotlyshare')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



PlotlyShare: New plot alleged sniffle 17th Jul created at https://plotlyshare-1-r8742502.deta.app/plot/3d90763ecafc9cbc2d4d4211b604438e of size 24.77kB


In [1]:
fig.update_layout(xaxis=dict(tickformat="%m-%d"))

# Plot cum dist over year
filtered_df = calendar_df[calendar_df['distance'] != 0]
fig = px.histogram(filtered_df, x='distance', color='year',
              title='Mileage Frequency',
              labels={'mthday': 'Date', 'distance': 'Distance (km)'})
fig.update_layout(xaxis=dict(tickformat="%b-%d"))


fig.show(renderer='plotlyshare')

NameError: name 'fig' is not defined